In [27]:
import os

In [28]:
%pwd


'd:\\Repositories\\Machine Learning\\ETE-Indian-Currency-Detection'

In [38]:
os.chdir("ETE-Indian-Currency-Detection")

In [39]:
%pwd

'd:\\Repositories\\Machine Learning\\ETE-Indian-Currency-Detection'

In [40]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_image_size: list
    params_classes: int

In [41]:
from CNNclassifier.constants.__init import CONFIG_FILE_PATH,PARAMS_FILE_PATH    
from CNNclassifier.utils.common import read_yaml,create_directories


In [42]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_image_size=tuple(self.params.IMAGE_SIZE),
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [43]:
prepare_base_model_config.params_image_size

BoxList([300, 300, 3])

In [44]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [45]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"TensorFlow has detected {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"- {gpu}")
else:
    print("No GPU detected by TensorFlow.")

TensorFlow has detected 1 GPU(s):
- PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [49]:
class PrepareBaseModel:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        """
        Constructs a sequential model and saves it to the specified path in the configuration.
        """

        # Build the sequential model
        self.model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=self.config.params_image_size),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.config.params_classes, activation='softmax')
        ])

        # Build the model by passing a dummy input shape
        self.model.build(input_shape=(None,) + self.config.params_image_size)

        # Display the model summary
        self.model.summary()

        # Save the model
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, learning_rate=None):
        """
        Adds custom layers to the base model for classification.
        """
        # Add custom layers for fine-tuning
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        # Create the full model
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        # Compile the full model
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        # Display the model summary
        full_model.summary()
        return full_model

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [51]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e


[2025-01-24 13:21:48,004: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-01-24 13:21:48,009: INFO: common: YAML file: params.yaml loaded successfully]
[2025-01-24 13:21:48,054: INFO: common: created directory at: artifacts]
[2025-01-24 13:21:48,057: INFO: common: created directory at: artifacts/prepare_base_model]
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_12 (Rescaling)    (None, 300, 300, 3)       0         
                                                                 
 conv2d_60 (Conv2D)          (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 149, 149, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 147, 14